In [24]:
!pip install tensorflow

  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata (5.1 kB)
  Obtaining dependency information for tensorboard<2.16,>=2.15 from https://files.pythonhosted.org/packages/37/12/f6e9b9dcc310263cbd3948274e286538bd6800fd0c268850788f14a0c6d0/tensorboard-2.15.2-py3-none-any.whl.metadata
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Obtaining dependency information for tensorflow-estimator<2.16,>=2.15.0 from https://files.pythonhosted.org/packages/b6/c8/2f823c8958d5342eafc6dd3e922f0cc4fcf8c2e0460284cc462dae3b60a0/tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Obtaining dependency information for keras<2.16,>=2.15.0 from htt

In [25]:
!pip install scikeras[tensorflow]

  Obtaining dependency information for tensorflow<2.13.0,>=2.12.0 from https://files.pythonhosted.org/packages/ca/a9/8cf654c2a89ea0103f3d3ea6f953b425d22f98736f7c2d2ebccf863b6d31/tensorflow-2.12.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.12.1-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Obtaining dependency information for tensorflow-intel==2.12.1 from https://files.pythonhosted.org/packages/d4/60/1ca023f19c8c22e283f5fd3fd209dbe11c8fb7a5078fc4ff247738eab74d/tensorflow_intel-2.12.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.12.1-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.12.1-cp311-cp311-win_amd64.whl (1.9 kB)
Using cached tensorflow_intel-2.12.1-cp311-cp311-win_amd64.whl (272.9 MB)
  Attempting uninstall: tensorflow-estimato

In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [2]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

OptionError: Pattern matched multiple keys

In [4]:
spotify_df = pd.read_csv('spotify_music.csv')

In [5]:
spotify_df.head()

,Unnamed: 0,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,0,Radhe Shyam,1,7Bnc9gWHdjZAmjSuTY42CJ,Aashiqui Aa Gayi,spotify:track:7Bnc9gWHdjZAmjSuTY42CJ,0.587,0.527,0.495,0.000000,0.127,-7.783,0.0353,122.826,0.474,53
1,1,Radhe Shyam,2,4hPxFf3Rd18BRtTYRcqZir,Soch Liya,spotify:track:4hPxFf3Rd18BRtTYRcqZir,0.484,0.597,0.520,0.000001,0.103,-7.624,0.0360,119.923,0.192,54
2,2,Radhe Shyam,3,7j1ITbGbqY40RmHMgpTWn2,Udd Jaa Parindey,spotify:track:7j1ITbGbqY40RmHMgpTWn2,0.510,0.555,0.631,0.009250,0.187,-9.910,0.0382,122.011,0.311,34
3,3,Radhe Shyam,4,2q4Lxk4MskxCtKjCZaL7IN,Jaan Hai Meri,spotify:track:2q4Lxk4MskxCtKjCZaL7IN,0.693,0.536,0.623,0.000002,0.137,-7.856,0.0743,79.915,0.398,42
4,4,Radhe Shyam,5,4B4hRyO842T5th9ucxPQGJ,Main Ishq Mein Hoon,spotify:track:4B4hRyO842T5th9ucxPQGJ,0.598,0.386,0.586,0.000002,0.103,-8.108,0.0474,132.975,0.190,36


In [5]:
spotify_df.dropna( inplace=True)

In [6]:
spotify_df.isna().sum()

Unnamed: 0          0
album               0
track_number        0
id                  0
name                0
uri                 0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [7]:
spotify_df.head()

,Unnamed: 0,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,0,Radhe Shyam,1,7Bnc9gWHdjZAmjSuTY42CJ,Aashiqui Aa Gayi,spotify:track:7Bnc9gWHdjZAmjSuTY42CJ,0.587,0.527,0.495,0.000000,0.127,-7.783,0.0353,122.826,0.474,53
1,1,Radhe Shyam,2,4hPxFf3Rd18BRtTYRcqZir,Soch Liya,spotify:track:4hPxFf3Rd18BRtTYRcqZir,0.484,0.597,0.520,0.000001,0.103,-7.624,0.0360,119.923,0.192,54
2,2,Radhe Shyam,3,7j1ITbGbqY40RmHMgpTWn2,Udd Jaa Parindey,spotify:track:7j1ITbGbqY40RmHMgpTWn2,0.510,0.555,0.631,0.009250,0.187,-9.910,0.0382,122.011,0.311,34
3,3,Radhe Shyam,4,2q4Lxk4MskxCtKjCZaL7IN,Jaan Hai Meri,spotify:track:2q4Lxk4MskxCtKjCZaL7IN,0.693,0.536,0.623,0.000002,0.137,-7.856,0.0743,79.915,0.398,42
4,4,Radhe Shyam,5,4B4hRyO842T5th9ucxPQGJ,Main Ishq Mein Hoon,spotify:track:4B4hRyO842T5th9ucxPQGJ,0.598,0.386,0.586,0.000002,0.103,-8.108,0.0474,132.975,0.190,36


In [8]:
mood_prep = spotify_df[[ 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo','popularity']]

In [9]:
col_features = mood_prep.columns[:]
col_features

Index(['danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo',
       'popularity'],
      dtype='object')

In [10]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [33]:
mood_trans_np

array([[5.27000e-01, 5.87000e-01, 4.95000e-01, ..., 3.53000e-02,
        1.22826e+02, 5.30000e+01],
       [5.97000e-01, 4.84000e-01, 5.20000e-01, ..., 3.60000e-02,
        1.19923e+02, 5.40000e+01],
       [5.55000e-01, 5.10000e-01, 6.31000e-01, ..., 3.82000e-02,
        1.22011e+02, 3.40000e+01],
       ...,
       [6.41000e-01, 4.62000e-01, 7.63000e-01, ..., 9.80000e-02,
        1.32987e+02, 3.20000e+01],
       [1.88000e-01, 6.59000e-01, 3.38000e-01, ..., 3.35000e-02,
        1.77647e+02, 2.50000e+01],
       [4.52000e-01, 5.98000e-01, 6.23000e-01, ..., 3.66000e-02,
        8.08300e+01, 3.90000e+01]])

In [12]:
mood_trans_np[100]#checking any value

array([ 6.9700e-01,  4.7200e-01,  6.6000e-01,  0.0000e+00,  2.4300e-01,
        3.0100e-01, -5.4590e+00,  3.5200e-02,  9.3011e+01,  4.0000e+01])

In [13]:
df = pd.read_csv('data_moods.csv')

In [18]:
df.columns

Index(['name', 'album', 'artist', 'id', 'release_date', 'popularity', 'length',
       'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo', 'key',
       'time_signature', 'mood'],
      dtype='object')

In [14]:
df.head()

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
0,1999,1999,Prince,2H7PHVdQ3mXqEHXcvclTB0,1982-10-27,68,379266,0.866,0.13700,0.730,0.000000,0.0843,0.625,-8.201,0.0767,118.523,5,4,Happy
1,23,23,Blonde Redhead,4HIwL9ii9CcXpTOTzMq0MP,2007-04-16,43,318800,0.381,0.01890,0.832,0.196000,0.1530,0.166,-5.069,0.0492,120.255,8,4,Sad
2,9 Crimes,9,Damien Rice,5GZEeowhvSieFDiR8fQ2im,2006-11-06,60,217946,0.346,0.91300,0.139,0.000077,0.0934,0.116,-15.326,0.0321,136.168,0,4,Sad
3,99 Luftballons,99 Luftballons,Nena,6HA97v4wEGQ5TUClRM0XLc,1984-08-21,2,233000,0.466,0.08900,0.438,0.000006,0.1130,0.587,-12.858,0.0608,193.100,4,4,Happy
4,A Boy Brushed Red Living In Black And White,They're Only Chasing Safety,Underoath,47IWLfIKOKhFnz1FUEUIkE,2004-01-01,60,268000,0.419,0.00171,0.932,0.000000,0.1370,0.445,-3.604,0.1060,169.881,1,4,Energetic


In [21]:
cl_features = col_features #features that are present in our dataset
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [22]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
5,Calm,0
4,Energetic,1
0,Happy,2
1,Sad,3


In [23]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [26]:
#Configure the model
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [27]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Baseline: 77.25% (2.80%)


In [29]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)

5/5 [==============================] - 0s 3ms/step


In [30]:
y_preds

array([1, 0, 0, 1, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 0, 3, 3, 1, 3, 2, 0, 0,
       3, 0, 1, 0, 3, 3, 3, 2, 1, 1, 0, 1, 3, 0, 2, 1, 0, 3, 1, 0, 0, 2,
       3, 3, 0, 2, 1, 0, 2, 1, 3, 0, 1, 3, 1, 1, 1, 1, 1, 0, 1, 0, 3, 1,
       3, 2, 0, 3, 1, 0, 3, 0, 0, 3, 3, 3, 1, 0, 3, 2, 2, 2, 0, 1, 3, 3,
       0, 1, 1, 0, 1, 2, 3, 1, 1, 0, 3, 1, 2, 0, 0, 3, 3, 2, 2, 0, 1, 1,
       0, 3, 0, 0, 2, 1, 2, 3, 3, 0, 3, 0, 2, 0, 1, 0, 0, 0, 0, 1, 3, 3,
       1, 1, 1, 3, 3, 2])

In [31]:
#Join the model and the scaler in a Pipeline
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#Fit the Pipeline
pip.fit(X2,encoded_y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('keras',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000254A383EE10>)])

In [32]:
def predict_mood(preds):
    
#     pipe = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#     #Fit the Pipeline
#     pipe.fit(X2,encoded_y)

    preds_features = np.array(preds[:]).reshape(-1,1).T

    #Predict the features of the song
    results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")
    

In [34]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 51ms/step


In [35]:
spotify_df.shape

(1586, 16)

In [36]:
print(len(res))

1586


In [37]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [38]:
spotify_df.to_csv('mood_wala.csv')

In [39]:
res.count("Sad")

689

In [40]:
res.count("Happy")

113

In [41]:
res.count("Energetic")

499

In [42]:
res.count("Calm")

285

In [43]:
spotify_df.shape

(1586, 17)

In [ ]:
spotify_df